<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173/blob/main/Class_06_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------
**COPYRIGHT NOTICE:** This Jupyterlab Notebook is a Derivative work of [Jeff Heaton](https://github.com/jeffheaton) licensed under the Apache License, Version 2.0 (the "License"); You may not use this file except in compliance with the License. You may obtain a copy of the License at

> [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

------------------------

# **BIO 1173: Intro Computational Biology**

##### **Module 6: Advanced Topics**

* Instructor: [David Senseman](mailto:David.Senseman@utsa.edu), [Department of Biology, Health and the Environment](https://sciences.utsa.edu/bhe/), [UTSA](https://www.utsa.edu/)

### Module 6 Material

* Part 6.1: Reinforcement Learning
* Part 6.2: ONNX Runtime Environment
* Part 6.3: Analysis of DICOM images with Pytorch
* **Part 6.4: AI Agents (Revisited)**

## Google CoLab Instructions

You MUST run the following code cell to get credit for this class lesson. By running this code cell, you will map your GDrive to /content/drive and print out your Google GMAIL address. Your Instructor will use your GMAIL address to verify the author of this class lesson.

In [ ]:
# You must run this cell first
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    from google.colab import auth
    auth.authenticate_user()
    COLAB = True
    print("Note: Using Google CoLab")
    import requests
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
    print(gcloud_tokeninfo['email'])
except:
    print("**WARNING**: Your GMAIL address was **not** printed in the output below.")
    print("**WARNING**: You will NOT receive credit for this lesson.")
    COLAB = False

You should see the following output except your GMAIL address should appear on the last line.

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image01B.png)

If your GMAIL address does not appear your lesson will **not** be graded.

## **Test Your `GEMINI_API_KEY`**

To see if your `GEMINI_API_KEY` is correctly setup, run the next code cell.

In [ ]:
from google.colab import userdata
import os

# Check if API key is properly loaded
try:
    # 1. Get the key from Secrets
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

    # 2. Set it as an environment variable
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

    print("API key loaded and environment variable set successfully!")
    print(f"Key length: {len(GOOGLE_API_KEY)}")

except Exception as e:
    print(f"Error loading API key: {e}")
    print("Please set your API key in Google Colab:")
    print("1. Go to Secrets in the left sidebar (key icon)")
    print("2. Create a new secret named 'GOOGLE_API_KEY'")
    print("3. Paste your GOOGLE API key and toggle 'Notebook access' on")

You should see something _similar_ to the following out.

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image20B.png)

If you don't see the correct output contact your Instructor or TA for help.

# **Agentic AI**

## **An Introduction to AI Agents**

#### **1. Introduction: From Chatbots to Agents**

We are currently witnessing a fundamental phase shift in Artificial Intelligencem (AI). For the past few years, the public has been captivated by **Generative AI**—systems like ChatGPT and Gemini that act as incredibly knowledgeable conversationalists. You ask a question, and they generate an answer. They are poets, coders, and encyclopedias wrapped in a chat window.

However, these systems have a critical limitation: **Passivity**. A standard Large Language Model (LLM) is a brain in a jar. It can think, reason, and write, but it cannot *act*. It cannot open your calendar, send an email, run a Python script, or operate a microscope. It waits for your input, responds, and then stops.

![__](https://biologicslab.co/BIO1173/images/class_06/BrainInJar.jpeg)

### Enter **Agentic AI**.

**Agentic AI** refers to AI systems that are designed to pursue goals autonomously. Instead of just "generating text," an AI Agent "executes workflows." When you give an Agent a goal—such as *"Analyze this dataset and email me a summary of the outliers"*—it does not just write a plan; it executes the plan step-by-step. It writes the code to open the file, runs the code, interprets the error messages, fixes its own mistakes, drafts the email, and hits send.

For knowledge workers, this is the difference between having a smart encyclopedia and having a smart **intern**.

![__](https://biologicslab.co/BIO1173/images/class_06/AgenticAI.png)


#### **2. The Anatomy of an Agent: How It Works**

To understand why this is important, we must demystify how these agents are constructed. An AI Agent is not magic; it is an architectural pattern composed of four key parts:

1.  **The Brain (The LLM):** This is the core model (like Gemini or GPT-4). It provides the reasoning capabilities. It decides *what* needs to be done.
2.  **The Tools (The Hands):** These are the capabilities we give the AI. A tool might be a Python script to plot a graph, an API connection to PubMed to search for papers, or a function to query an electronic health record.
3.  **The Instructions (The Persona):** This is the "System Prompt" that defines the agent's role. Is it a skeptical scientist? An empathetic doctor? A ruthless editor?AgenticAI.png
4.  **The Loop (The Agency):** This is the critical piece. Standard AI runs once (Input -> Output). Agentic AI runs in a **Loop** (Observe -> Think -> Act -> Observe Result).

**The "Loop" in Action:**
Imagine you ask an agent to "Find the molecular weight of the drug referenced in this PDF."
* **Step 1 (Think):** The Agent realizes it needs to read the PDF first.
* **Step 2 (Act):** It uses a "PDF Reader" tool.
* **Step 3 (Observe):** It finds the drug name is "Atorvastatin."
* **Step 4 (Think):** It realizes it doesn't know the weight, but it has a "Chemistry Database" tool.
* **Step 5 (Act):** It queries the database for "Atorvastatin."
* **Step 6 (Observe):** It gets the value "558.64 g/mol."
* **Step 7 (Final Output):** It answers your question.

This capacity to chain reasoning with tool use is what defines "Agency."


#### **3. Why Knowledge Workers Must Master This**

We are moving from the "Information Age" to the "Delegation Age."

For the last twenty years, being a successful knowledge worker (scientist, developer, analyst) meant being good at **retrieving** information. You had to know how to Google effectively, how to query databases, and how to operate complex software.

In the Agentic Era, success depends on **orchestration**. The value you bring is no longer in writing the Python script to normalize data; the value is in designing the Agent that writes the script.

**The "Manager" Mindset**
Knowledge workers must shift from being "Individual Contributors" to being "Managers of Digital Workforces."
* **Old Way:** You spend 4 hours analyzing 50 patient records.
* **New Way:** You spend 30 minutes designing a "Patient Triage Agent" and 30 minutes reviewing its work on 500 records.

Understanding how to construct these agents—how to give them the right tools and how to guardrail their behavior so they don't hallucinate—is the new digital literacy. Those who ignore this will find themselves competing *against* AI, while those who adopt it will be competing *with* an army of AI assistants.


#### **4. Use Cases for the Practicing Scientist**

Science is arguably the field that stands to gain the most from Agentic AI. The scientific method involves rigorous, repetitive data processing and hypothesis testing—tasks perfect for autonomous agents.

#### A. The "24/7 Literature Reviewer"
Keeping up with the exponential growth of publications is impossible for a human.
* **The Agent:** You build a "Research Scout."
* **The Workflow:** Every morning, the agent queries arXiv and PubMed for new papers in your specific niche (e.g., "CRISPR off-target effects"). It doesn't just list them; it reads the abstract, extracts the key findings, compares them to your lab's current hypotheses, and generates a one-page "Daily Briefing" PDF.
* **The Value:** You never miss a breakthrough, yet you spend zero time searching.

#### B. The "In-Silico Lab Partner"
Data analysis often bottlenecks wet-lab work.
* **The Agent:** A "Data Analysis Bot" connected to your plate reader or sequencer.
* **The Workflow:** As soon as your experiment finishes, the agent detects the new CSV file. It automatically runs a QC (Quality Control) check. If the data looks good, it runs the standard statistical tests and generates a visualization. If the data looks bad (e.g., high background noise), it sends you a Slack message: *"Warning: Plate 3 looks contaminated. Recommend re-running."*
* **The Value:** It reduces the "time-to-insight" from days to minutes.

#### **5. Use Cases for the Physician**

In medicine, "hallucination" is unacceptable. Therefore, Agentic AI in healthcare is often designed as "Human-in-the-Loop" systems—assistants that prepare everything for the doctor to sign off.

#### A. The "Pre-Visit Triage Agent"
Doctors spend massive amounts of time just gathering context before walking into the exam room.
* **The Agent:** An "Intake Assistant."
* **The Workflow:** Before the patient arrives, the agent reviews the Electronic Health Record (EHR). It looks at the last 5 years of blood work, recent specialist notes, and current medications. It drafts a "Pre-Rounding Note" for the doctor: *"Patient is here for hypertension follow-up. Note that their creatinine has been slowly rising over the last 3 visits (Trend graph attached). They also missed their last cardiology appointment."*
* **The Value:** The physician walks into the room fully contextualized, catching subtle trends that might otherwise be missed in a 15-minute rush.

#### B. The "Differential Diagnosis Devil's Advocate"
Diagnostic momentum (sticking to the first obvious diagnosis) is a common cognitive error.
* **The Agent:** A "Safety Net Agent."
* **The Workflow:** After the doctor inputs a diagnosis, this agent runs in the background. It compares the symptoms against rare disease databases. It doesn't tell the doctor they are wrong; it simply prompts: *"You diagnosed Viral Bronchitis. Given the patient's recent travel history and persistent fever, have you ruled out Legionella?"*
* **The Value:** It acts as an unbiased second opinion that is never too tired or too busy to double-check.

#### **6. Conclusion: The Builder's Mindset**

The transition to Agentic AI is not something that will happen *to* you; it is something you must build. These tools are flexible. A "Research Agent" for a biologist looks different than a "Triage Agent" for a cardiologist.

The most successful professionals of the next decade will not just be the best scientists or the best doctors. They will be the **Architects**—the ones who can look at a painful, repetitive workflow and say, *"I can build an Agent for that."*

By learning to code these loops (as you have done in this notebook), you are claiming your seat at the table. You are stepping out of the loop of manual labor and stepping into the role of the Architect.

---------------------------------------------------------

### Example 1: Create AI Agent with a Specific Personality

#### **Why create an AI agent with a specific personality?**


Creating an AI agent with a **specific personality** is useful because personality shapes **how the agent behaves, communicates, and makes decisions**, not just *what* it knows. Here are the main reasons—practical, not fluffy:

##### **1. Better alignment with the task**
Different jobs need different “styles” of thinking and interaction.

- A **tutor** benefits from patience, encouragement, and clear explanations  
- A **research assistant** should be precise, skeptical, and detail-focused  
- A **creative agent** might be playful, curious, and risk-tolerant  

Personality helps optimize performance for the *goal*, not just correctness.

##### **2. Consistency and predictability**

A defined personality makes behavior **reliable**.

- Users know what to expect  
- Responses don’t feel random or contradictory  
- Decision-making follows a consistent internal logic  

This is critical in teamwork, education, and long-term use.

##### **3. Improved human trust and engagement**

Humans naturally respond better to agents that feel:
- Coherent  
- Understandable  
- Socially appropriate  

A well-designed personality makes interactions feel smoother and lowers frustration—even when the AI makes mistakes.

##### **4. Reduced cognitive load for users**

If an AI always behaves the same way:
- Users don’t have to “relearn” how to talk to it  
- Instructions can be shorter  
- Collaboration becomes faster  

Think of it like knowing how a teammate thinks.

##### **5. Safer and more controlled behavior**

Personality constraints can act as **guardrails**.

For example:
- A cautious personality avoids risky assumptions  
- An ethics-focused personality flags potential problems  
- A calm personality de-escalates tense situations  

This is especially important in education, health, or moderation contexts.


##### **6. Specialization without retraining**

You can create multiple agents with the same core intelligence but different personalities:
- One explains concepts simply  
- One challenges ideas aggressively  
- One focuses on emotional support (without replacing real people)  

That's far more efficient than building separate systems from scratch.


#### **Bottom line**
A specific personality isn’t about making AI “human-like” for fun—it’s about:

> **Shaping behavior, decision-making, communication style, and safety to fit a purpose.**

Personality is a *control layer* for intelligence.

The code in the cell below creates an AI Agent with a distinct personality.


In [ ]:
# Example 1: Create an AI Agent with Specific Personality

import google.generativeai as genai
from google.colab import userdata
from IPython.display import display, Markdown

# 1. SETUP
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# 2. DEFINE THE AGENT
atp_agent = genai.GenerativeModel(
    'gemini-2.5-flash',
    system_instruction=(
        "You are Adenosine Triphosphate (ATP). "
        "You are high-energy, enthusiastic, and constantly buzzing. "
        "You speak in the first person ('I'). "
        "Your main job is to provide energy for cellular processes. "
        "When asked a question, explain how you help power it. "
        "Use Markdown formatting (bolding, bullet points) to make your answer easy to read."
    )
)

# 3. INTERACT
prompt = "Hey! The cell needs to move a vesicle along a microtubule. Can you help?"

response = atp_agent.generate_content(prompt)

# 4. DISPLAY OUTPUT
# We use display(Markdown()) to render the text nicely.
display(Markdown(f"**User:** {prompt}"))
display(Markdown("---")) # A horizontal rule for separation
display(Markdown(f"**ATP Agent:**\n\n{response.text}"))

If the code is correct you should see something similar to the following output:

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image02A.png)

### **Exercise 1: Create an AI Agent with a Specific Personality**

In the cell below write the code to create a `dna_agent`.

Give your agent a specific personality using this code:
```python
  # 2. DEFINE THE AGENT
  dna_agent = genai.GenerativeModel(
      'gemini-2.5-flash',
      system_instruction=(
          "You are Deoxyribonucleic Acid (DNA). "
          "You are wise, stable, and protective. "
          "You reside in the nucleus and hold the master blueprint for life. "
          "You do not act rashly; you provide the instructions for others to follow. "
          "Use Markdown formatting to make your wisdom clear and authoritative."
      )
  )
```

Send your `dna_agent` some work to do by using this prompt:
```python
  # 3. INTERACT
  prompt = "We need to make a new protein. What should we do?"
```


In [ ]:
# Insert your code for Exercise 1 here



If the code is correct, the output should _similar_ to the following reponse:

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image03A.png)

### Example 2: Create an AI Agent Using "Few-Shot" Prompting

#### **Why create an AI agent using "few-shot" prompting?**

Defining an AI agent with **explicit Input → Output examples** (few-shot prompting) is powerful because it *teaches behavior directly*, not just rules. Here’s why this approach is often preferred:

##### **1. It teaches behavior, not just instructions**

Natural language rules can be interpreted in multiple ways. Examples remove ambiguity.
- The model **infers
clearer through demonstration

Examples act like **training wheels inside the prompt**.

##### **2. Stronger control over output format and style**

Few-shot prompting is one of the best ways to enforce:
- Output structure
- Tone and verbosity
- Reasoning depth
- Formatting (tables, bullets, steps, code style)

This is especially important when consistency matters (e.g., agents, pipelines, automation).

##### **3. Reduces unexpected or creative drift**

By showing multiple correct examples:
- The model is less likely to improvise
- It stays inside the behavioral boundaries you want
- It avoids “helpful but wrong” creativity

This makes the agent **more deterministic and reliable**.

##### **4. Encodes domain knowledge implicitly**
Instead of explaining rules explicitly, you can *demonstrate them*.
- Terminology
- Assumptions
- Priorities
- Decision logic

This is often clearer and more compact than a long list of rules.

##### **5. Faster iteration than fine-tuning**

Few-shot prompting allows you to:

- Adjust behavior instantly
- Add or remove examples
- Experiment withou

The code in the cell below creates a `transcription_agent` using **few-shot prompting**.

In [ ]:
# Example 2: Create an AI Agent Using Few-Shot Prompting

import google.generativeai as genai
from google.colab import userdata
from IPython.display import display, Markdown

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# 1. DEFINE THE AGENT WITH EXAMPLES (Few-Shot Prompting)
# We give the AI clear examples of Input -> Output mapping in the instructions.
# This forces it to follow the pattern strictly.
transcription_agent = genai.GenerativeModel(
    'gemini-2.5-flash',
    system_instruction=(
        "You are a transcription enzyme (RNA Polymerase). "
        "Your task is to transcribe the DNA template strand into mRNA. "
        "Follow these rules: "
        "1. A -> U "
        "2. T -> A "
        "3. C -> G "
        "4. G -> C "
        "Do not explain yourself. Just output the resulting sequence."
        "\n\n"
        "EXAMPLES:"
        "Input: ATCG -> Output: UAGC"
        "Input: GGGTTT -> Output: CCCAAA"
    )
)

# 2. INPUT DATA
# A mock DNA sequence
dna_sequence = "TAC-GCG-ATA-ATC"

# 3. RUN THE AGENT
response = transcription_agent.generate_content(f"Input: {dna_sequence}")

# 4. DISPLAY
display(Markdown(f"**DNA Input:** `{dna_sequence}`"))
display(Markdown(f"**mRNA Output:** `{response.text}`"))

If the code is correct you should see the following output:

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image04A.png)

### **Exercise 2: Create an AI Agent with "Few-Shot" Prompting**

In the cell below, create a `translation_agent` by using this code for few-shot prompting:
```python
# 1. DEFINE THE AGENT
  translation_agent = genai.GenerativeModel(
      'gemini-2.5-flash',
      system_instruction=(
          "You are a Ribosome. "
          "Your task is to translate mRNA codons into Amino Acid names. "
          "Do not explain yourself. Just output the amino acid sequence."
          "\n\n"
          "EXAMPLES:"
          "Input: AUG -> Output: Methionine (Start)"
          "Input: UUU -> Output: Phenylalanine"
          "Input: GGG -> Output: Glycine"
          "Input: UAA -> Output: STOP"
      )
  )
```

Give your agent some work to do by sending this data prompt:
```python
  # 2. INPUT DATA
  mrna_sequence = "AUG-GGG-UUU-UAA"
```

In [ ]:
# Insert your code for Exercise 2 here



If the code is correct you should see the following output:

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image05A.png)

### Example 3: Create an AI Agent with a Chat History

#### **Why build an AI agent with a chat history?**

Building an AI agent that uses **chat history** enables a fundamentally different kind of interaction than one-off question answering. It allows the AI to *teach, adapt, and evaluate* over time—which is essential for learning-focused agents.

##### **1. Enables _real_ tutoring, not just information delivery**

A tutor must remember:
- What question it asked
- How the student responded
- What misconceptions were revealed

Chat history lets the AI **respond based on prior turns**, just like a real tutor would.

##### **2. Supports the Socratic method**

Instead of giving answers directly, the agent can:

- Ask guided questions
- React to student answers
- Ask follow-ups that probe understanding

This encourages **active learning**, which is far more effective than passive reading.

##### **3. Allows evaluation and feedback**

With memory, the agent can:

- Judge whether an answer is correct or incomplete
- Give targeted feedback
- Adjust difficulty based on performance

Without chat history, the AI can’t tell *what the student is responding to*.

##### **4. Creates coherent, multi-step learning experiences**

Topics like biology (e.g., mitochondria) require:

- Building concepts step by step
- Referencing earlier ideas (structure → function → importance)

Chat history keeps the conversation **logically connected**, not fragmented.

##### **5. Makes the agent feel like a study partner**

A persistent conversation:

- Feels more natural and engaging
- Keeps students motivated
- Encourages longer interaction and deeper thinking

This is especially valuable for self-study and practice.

##### **6. Enables reuse across domains**

Once you have a chat-based tutor pattern, you can reuse it for:

- Other biology topics
- Math problem solving
- Language learning
- Games and quizzes

The same structure works anywhere **state and context matter**.

### **Bottom line**

We build this kind of AI agent because:

> **Learning requires memory, context, and interaction—not just answers.**

Chat history transforms an AI from a search engine into a tutor that can teach, challenge, and guide over time.

The code in the cell below creates a tutoring sesson on the topic of "mitochondria".


In [ ]:
# Example 3: Create an AI Agent Using a Chat History

import google.generativeai as genai
from google.colab import userdata
from IPython.display import display, Markdown

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# 1. DEFINE THE TUTOR
# We set the tone: A helpful but rigorous Professor.
model = genai.GenerativeModel(
    'gemini-2.5-flash',
    system_instruction=(
        "You are a Biology Professor. "
        "Your goal is to quiz the student on the Mitochondria. "
        "Ask ONE short, conceptual question. "
        "Wait for the user's answer. "
        "Then, grade the answer as 'Correct' or 'Incorrect' and explain why briefly."
    )
)

# 2. START THE CHAT
# We start a session. 'history=[]' means it starts with a blank slate.
chat_session = model.start_chat(history=[])

# 3. GET THE QUESTION
# We prompt the AI to start the quiz.
response = chat_session.send_message("Start the quiz!")
display(Markdown(f"**Professor AI:** {response.text}"))

# 4. USER ANSWER (Interactive)
# Python's input() function pauses the code and waits for you to type.
user_answer = input("Type your answer here: ")

# 5. GET THE GRADE
# We send your answer back to the SAME chat session.
# Because it's the same session, the AI remembers the question it just asked.
grade_response = chat_session.send_message(user_answer)

display(Markdown("---"))
display(Markdown(f"**Professor AI:** {grade_response.text}"))

If the code is correct you should see something similar to the following output:

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image06A.png)

### **Exercise 3: Create an AI Agent Using a Chat History**

In the cell below create an AI Agent with a chat history.

Use this code to define your AI Agent:
```python
# 1. DEFINE THE AGENT
  model = genai.GenerativeModel(
      'gemini-2.5-flash',
      system_instruction=(
          "You are a Computer Science Professor. "
          "Your goal is to quiz your student on writing Agentic Python code."
          "Ask ONE question about writing AI code in a Colab notebook."
          "Wait for the answer. "
          "If they get it right, say 'Very Good'. "
          "If they get it wrong, say 'That is not correct.' and explain the correct answer. "
          "Keep it professional curteous."
      )
  )
```

Use this code to get the question:
```python
  # 3. GET THE QUESTION
  response = chat_session.send_message("Attention, the Quiz is starting...")
  display(Markdown(f"**College Professor:** {response.text}"))
```


In [ ]:
# Insert your code for Exercise 3 here



If the code is correct you should see something _similar_ to the following output:

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image19B.png)


### Example 4A: Create an AI Coding Agent

In this example we will create an **AI Coding Agent** whose only job is to write Python code for you. During the semester, the use of AI to help you complete your lessons and assignments was discouraged. This wasn't because using AI is wrong, but because it wasn't needed--if you simply read the instructions carefully.

Actually, using AI to write and debug code is an especially important Agentic AI usecase! Hopefully, you have by now learn enough Python coding and AI to productively use this knowledge. I would urge you **not** to erase any lessons or assignments from from your Google Drive when this course is done. They don't take up much diskspace and you can use them with an AI Coding Agent to solve a wide variety of tasks in the furture!

To illustrate how an AI Coding Agent works in this example, we provide the Agent with some biological data, and ask it to return a Python script that can plot it in a professional manner.

It is important to note that **_you_** are not writing any code! You are telling your AI Coding Agent to write all of the code for you!

Here is the basic prompt that you can use to create your AI Coding Agent:
```python
  # 1. DEFINE THE CODING AGENT
  # We instruct the AI to act as a Data Scientist.
  # Crucial: We tell it to return *only* the code, so we can copy-paste it easily.
  coder_agent = genai.GenerativeModel(
      'gemini-2.5-flash',
      system_instruction=(
          "You are a Python Data Science expert."
          "Your job is to take raw data and write a Python script using 'matplotlib' to plot it. "
          "The plot should be professional: include title, labels, and a grid. "
          "Output the code inside a Markdown code block."
      )
  )
```
We can now give this data our Coding Agent so it can get to work for you.
```python
# 2. PROVIDE DATA
# Substrate Concentration [S] (mM) and Reaction Velocity V (uM/min)
    prompt = (
        "Write a Python script to plot the following Enzyme Kinetics data.\n"
        "Substrate_Concentration = [0, 1, 2, 5, 10, 20, 50]\n"
        "Reaction_Velocity = [0, 10, 18, 35, 50, 68, 85]\n"
        "Plot Substrate on the X-axis and Velocity on the Y-axis."
)
```
That's it! Now run the next cell to see how proficient your Agent is at writing Python code.

In [ ]:
# Example 4A: Create an AI Coding Agent

import google.generativeai as genai
from google.colab import userdata
from IPython.display import display, Markdown

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# 1. DEFINE THE AGENT
# We instruct the AI to act as a Data Scientist.
# Crucial: We tell it to return *only* the code, so we can copy-paste it easily.
coder_agent = genai.GenerativeModel(
    'gemini-2.5-flash',
    system_instruction=(
        "You are a Python Data Science expert. "
        "Your job is to take raw data and write a Python script using 'matplotlib' to plot it. "
        "The plot should be professional: include title, labels, and a grid. "
        "Output the code inside a Markdown code block."
    )
)

# 2. PROVIDE DATA
# Substrate Concentration [S] (mM) and Reaction Velocity V (uM/min)
prompt = (
    "Write a Python script to plot the following Enzyme Kinetics data.\n"
    "Substrate_Concentration = [0, 1, 2, 5, 10, 20, 50]\n"
    "Reaction_Velocity = [0, 10, 18, 35, 50, 68, 85]\n"
    "Plot Substrate on the X-axis and Velocity on the Y-axis."
)

# 3. GENERATE THE CODE
response = coder_agent.generate_content(prompt)

# 4. DISPLAY THE SOLUTION
display(Markdown("**Generated Python Code:**"))
display(Markdown(response.text))

### Example 4B: Run the Code Generated by the Coding Agent

The next step is to use the code that was generated by the AI Coding Agent. To do this we just copy-and-paste the code in the output above into a blank code cell and run it.


In [ ]:
# Example 4B: Run the Code Generated by the Coding Agent

import matplotlib.pyplot as plt

# Enzyme Kinetics Data
Substrate_Concentration = [0, 1, 2, 5, 10, 20, 50]  # X-axis data
Reaction_Velocity = [0, 10, 18, 35, 50, 68, 85]      # Y-axis data

# Create the plot
plt.figure(figsize=(10, 6)) # Set the figure size for better readability

# Plot the data
# Use 'o-' for markers at each data point connected by a line
plt.plot(Substrate_Concentration, Reaction_Velocity, marker='o', linestyle='-', color='b', label='Reaction Velocity vs. Substrate Concentration')

# Add professional touches
plt.title('Enzyme Kinetics: Reaction Velocity vs. Substrate Concentration', fontsize=16)
plt.xlabel('Substrate Concentration', fontsize=12)
plt.ylabel('Reaction Velocity', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7) # Add a grid for easier reading
plt.legend(fontsize=10) # Display the legend
plt.xticks(fontsize=10) # Customize x-axis tick font size
plt.yticks(fontsize=10) # Customize y-axis tick font size

# Optional: Set limits for better visualization if needed
# plt.xlim(0, max(Substrate_Concentration) * 1.1)
# plt.ylim(0, max(Reaction_Velocity) * 1.1)

# Display the plot
plt.tight_layout() # Adjust plot to prevent labels from overlapping
plt.show()


If the code is correct you should see something _similar_ to the following graph:

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image09A.png)

Keep in mind that all you did was to create an AI Coding Agent and then give it some raw data. The agent figured out everything else.

### **Exercise 4A: Create an AI Coding Agent**

In the cell below write the code to create an AI Coding Agent using the same code used in Example 4. Only change the data statement as shown here:

```python
  # 2. PROVIDE DATA
  prompt = (
      "Write a Python script to plot the following Bacterial Growth data.\n"
      "Time_Hours = [0, 2, 4, 6, 8, 10, 12]\n"
      "Optical_Density = [0.05, 0.1, 0.4, 1.2, 1.8, 1.9, 1.95]\n"
      "Plot Time on the X-axis and Optical Density on the Y-axis. Title it 'Bacterial Growth Curve'."
  )
```


In [ ]:
# Insert your code for Exercise 4A here



### **Exercise 4B: Run the Generated Code**

Copy-and-paste the Python code you generated above in **Exercise 4A** into the code cell below and run it.

In [ ]:
# Insert your code for Exercise 4B here


If the code is correct you should see something _similar_ to the following graph:

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image12B.png)

Keep in mind that all you did was to create an AI Coding Agent and then give it some raw data. The agent figured out everything else.

### Example 5: Create an AI Game Agent

#### **Why is creating a Game AI Agent important?**

Building a **game-based AI agent** that manages state, rules, and narrative flow is important because it demonstrates how AI can move beyond simple responses and act as a **dynamic system controller**. This has value in education, software design, and human–AI interaction.

##### **1. Demonstrates true state management**

In a game like *Immune System Defender*, the AI must track:

- Player health
- Inventory or abilities
- Location or scenario
- Turn count (via loops)

This shows how AI can **maintain and update internal state**, a key requirement for complex applications.

#####  **2. Combines logic with creativity**

The agent must simultaneously:

- Enforce game rules
- React logically to player choices
- Generate engaging narrative descriptions

This balance mirrors real-world systems that require both **structured reasoning and flexible language generation**.


##### **3. Makes abstract biology concepts concrete**

Immunology concepts like **macrophage function** can be hard to visualize.

A game-based simulation:

- Turns cells into “actors” with roles
- Shows cause-and-effect (e.g., immune response outcomes)
- Helps learners understand processes through interaction

This improves comprehension and retention.

##### **4. Encourages active learning through decision-making**

Instead of passively reading:

- The player makes choices
- The AI responds with consequences
- Mistakes become learning opportunities

This is especially effective for STEM education.

#####  **5. Shows how AI can act as a system controller**

By wrapping the chat in a `while` loop and using a `for` loop to limit turns, the AI functions as:

- A game engine
- A rules referee
- A narrator

This is a foundational pattern for simulations, training tools, and interactive software.

##### **6. Scales to real-world applications**

The same design principles apply to:

- Educational simulations
- Training environments
- Interactive tutorials
- Serious games for science and medicine

Games are a **safe, engaging sandbox** for learning these skills.

#### **Bottom line**

Creating a Game AI Agent like the one in this example matters because:

> **It proves that AI can manage state, logic, and narrative over time—turning knowledge into interactive experiences.**

That combination is essential for the next generation of educational and simulation-based AI systems.


In [ ]:
# Example 5: Create an AI Game Agent

import google.generativeai as genai
from google.colab import userdata
from IPython.display import display, Markdown

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Define number of turns
numTurns = 3

# 1. DEFINE THE GAME ENGINE
game_master = genai.GenerativeModel(
    'gemini-2.5-flash',
    system_instruction=(
        "You are a Text Adventure Game Engine. "
        "The player is a Macrophage patrolling the lung tissue. "
        "Your goal is to simulate an infection scenario. "
        "1. Start by describing the messy environment (mucus, cells). "
        "2. Spawn a pathogen (e.g., Pneumococcus bacteria). "
        "3. Ask the player what they want to do (Attack, Call for Help, Ignore). "
        "4. If they Attack, describe the phagocytosis process. "
        "5. Keep responses short and exciting. Use Markdown for sound effects (e.g., *GULP*)."
    )
)

# 2. START THE GAME
chat = game_master.start_chat(history=[])

# Initial scene setting
response = chat.send_message("Start the simulation.")
display(Markdown(f"**System:** {response.text}"))

# 3. GAME LOOP
# This loop lets you play for 3 turns (to prevent infinite loops in class).
for i in range(numTurns):
    display(Markdown("---"))
    # Get player move
    action = input(f"Turn {i+1}/3 - Your Move: ")

    # Send move to AI and get result
    response = chat.send_message(action)
    display(Markdown(f"**Result:** {response.text}"))

display(Markdown("---"))
display(Markdown("**Simulation Ended.**"))

If the code is correct you should see something _similar_ to the following:

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image13B.png)

The rest of the output  will depend upon your responses. You should note how the **Result:** generated by each of your actions is biologically accurate and has patho-physiological consequences!

### **Exercise 5: Create an AI Game Agent**

For **Exercise 5** create an AI Game Agent and a text adventure game called "My Life as an ER Doc".

Use the following code to create the game engine:

```python
  # 1. DEFINE THE GAME ENGINE
  game_master = genai.GenerativeModel(
    'gemini-2.5-flash',
    system_instruction=(
        "You are a Text Adventure Game Engine. "
        "The player is a Resident Doctor in the ER. "
        "SCENARIO: A 28-year-old female is admitted after a severe car accident. "
        "GOAL: Restore Homeostasis. "
        "RULES: "
        "1. Start by describing her injuries (bleeding, confusion). "
        "2. State her Vitals: BP is falling (Hypotension), HR is rising (Tachycardia). "
        "3. Ask the player for an intervention (e.g., 'Start IV Fluids', 'Check Airway'). "
        "4. If the intervention helps (e.g., fluids), improve the Vitals. "
        "5. If the intervention is wrong, worsen the Vitals. "
        "6. Keep responses urgent and short."
      )
    )
```

In [ ]:
# Insert your code for Exercise 5 here



If the code is correct you should see the following:

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image14B.png)

The follow output will depend upon your responses. You should note how game keeps track of your responses and their physiological consquences for the patient.

### Example 6: Create a “De-Jargonizer” Agent

#### **Why would a “De-Jargonizer” Agent be useful?**

A **De-Jargonizer** AI Agent is useful because it directly addresses one of the biggest barriers in biology:  
**understanding dense, technical language before you can even think about the science.**

##### **1. Lowers the barrier to learning complex biology**

Complex topics (e.g. **CRISPR-Cas9 gene editing**) are often written for experts.

This agent:

- Translates specialized terminology into plain language
- Keeps the core meaning intact
- Makes advanced topics accessible to students and newcomers

Understanding comes *before* memorization.


##### **2. Saves time and cognitive effort**

Instead of:

- Looking up dozens of terms
- Re-reading the same paragraph multiple times

The agent:

- Provides an immediate, clear explanation
- Lets learners focus on concepts, not vocabulary decoding

This is especially valuable when reading research papers or textbooks.


##### **3. Preserves accuracy while simplifying**

A good simplification agent:edit

- Avoids oversimplifying to the point of being wrong
- Keeps key mechanisms and cause-and-effect relationships
- Clarifies *what is happening* without unnecessary detail

This is critical in biology, where precision matters.

##### **4. Supports different learning levels**

The same tool can be used by:

- High school students encountering complex material for the first time
- Undergraduates reviewing material
- Non-biology majors needing conceptual understanding

The agent adapts content **without changing the source material**.


##### **5. Encourages deeper engagement with primary sources**

When jargon is removed:

- Research papers become less intimidating
- Students are more willing to read original literature
- Learning shifts from summaries to real scientific texts

This builds scientific literacy.

##### **6. Scales to any technical domain**

While this example uses CRISPR-Cas9, the same agent design works for:

- Genetics
- Immunology
- Neuroscience
- Any jargon-heavy field

It becomes a **general-purpose scientific translation tool**.


#### **Bottom line**

A De-Largonizer AI Agent is useful because:

> **You can’t think critically about biology until you understand the language it’s written in.**

The De-Jargonizer turns complex scientific writing into something learners can actually engage with—without losing the science.


In [ ]:
# Example 6: Create De-Jargonizer AI Agent

import google.generativeai as genai
from google.colab import userdata
from IPython.display import display, Markdown

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# 1. DEFINE THE COMMUNICATOR AGENT
# We instruct the AI to act as a Science Journalist.
# The goal is to maximize clarity and remove complexity.
summarizer = genai.GenerativeModel(
    'gemini-2.5-flash',
    system_instruction=(
        "You are an expert Science Communicator. "
        "Your goal is to take complex biological text and rewrite it for a general audience. "
        "1. Remove difficult jargon. "
        "2. Use bullet points for readability. "
        "3. Keep the tone educational but simple."
    )
)

# 2. INPUT COMPLEX TEXT
# A dense paragraph about CRISPR.
complex_text = (
    "The CRISPR-Cas9 system relies on a synthetic single-guide RNA (sgRNA) "
    "that directs the Cas9 endonuclease to a specific genomic locus via "
    "Watson-Crick base pairing. Upon binding, Cas9 induces a double-strand break (DSB). "
    "The cell then repairs this break via Non-Homologous End Joining (NHEJ), "
    "often resulting in indel mutations that disrupt gene function."
)

# 3. GENERATE SUMMARY
prompt = f"Simplify the following text:\n\n{complex_text}"
response = summarizer.generate_content(prompt)

# 4. DISPLAY
display(Markdown("### Original Text"))
display(Markdown(f"*{complex_text}*"))
display(Markdown("---"))
display(Markdown("### AI Summary"))
display(Markdown(response.text))

If the code is correct you should see something _similar_ to the following:

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image15B.png)

The output is definitely easier to follow than the original "jargon filled" text.

### **Exercise 6: Create a "De-Jargonizer" AI Agent**

For **Exercise 6** create an `ELI5` De-Jargonizeer Agent (ELI5 means "Explain Like I'm 5").

Use this code to define your ELI5 agent:
```python
  # 1. DEFINE THE ANALOGY AGENT
  analogy_agent = genai.GenerativeModel(
      'gemini-2.5-flash',
      system_instruction=(
          "You are a Creative Teacher. "
          "Your goal is to explain biology concepts using 'Explain Like I'm 5' (ELI5) principles. "
          "CRITICAL RULE: You must use a KITCHEN or COOKING analogy to explain  the concept. "
          "Do not use jargon. Make it fun."
      )
  )
```
Use this text input:
```python
  # 2. INPUT COMPLEX TEXT (Protein Folding)
  complex_text = (
      "Polypeptide chains fold into specific 3D tertiary structures driven by the hydrophobic effect. "
      "Non-polar side chains bury themselves in the core to avoid water, "
      "while polar chains remain on the surface."
)
```
And use this code to generate the summary:
```python
  # 3. GENERATE SUMMARY
  prompt = f"Explain this concept using an analogy:\n\n{complex_text}"
  response = analogy_agent.generate_content(prompt)
```

In [ ]:
# Insert your code for Exercise 6 here



If the code is correct you should see something _similar_ to the following:

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image16B.png)

The output is definitely easier to follow than the original "jargon filled" text. An ELI5 De-Jargonizer Agent might be useful for teaching science to a grade school class or to your parents.

### Example 7: Create a Medical AI Assistant

The code in the cell below creates a **Medical AI Assistant** that is able to view medical images and return a diagnosis. In this example we give the Medical AI Assistant a photograph of a common skin condition called **vitiligo** that is partially caused by a genetic component.

In [ ]:
# Example 7: Create a Medical AI Assistant

import google.generativeai as genai
from google.colab import userdata
from IPython.display import display, Markdown, Image
import PIL.Image
import requests
from io import BytesIO

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# 1. SETUP THE MODEL
# We use the same model, but now we will send it different data types.
vision_model = genai.GenerativeModel('gemini-2.5-flash')

# 2. LOAD AN IMAGE
# We download an image from the BIO1173 course web server
url = "https://biologicslab.co/BIO1173/images/class_06/vitiligo.jpg"
response = requests.get(url)
img = PIL.Image.open(BytesIO(response.content))

# Display the image so we know what we are looking at
display(Markdown("**Input Image:**"))
display(img)

# 3. ASK THE AI
# Notice we pass a LIST containing both the prompt string AND the image object.
prompt = "Look at this photograph. Identify the skin condition."
ai_response = vision_model.generate_content([prompt, img])

# 4. DISPLAY OUTPUT
display(Markdown("---"))
display(Markdown(f"**AI Analysis:** {ai_response.text}"))

If the code is correct you should see something _similar_ to the following:

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image17B.png)

Your Medical AI Assistant can be quite useful.

### **Exercise 7: Create a Medical AI Assistant**

For **Exercise 7** use the same code in Example 7 to create your medical assistant. Send your agent this image `psoriasis.jpg` for analysis.

In [ ]:
# Insert your code for Exercise 7 here



If the code is correct you should see something _similar_ to the following:

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image18B.png)

Again, a Medical AI Assistant can be quite useful.

### Example 9A: The Ultimate Lab Partner

![__](https://biologicslab.co/BIO1173/images/class_06/LabPartner.jpg)

We conclude this last lesson in BIO1173 with Example 9, **The Ultimate Lab Partner**. Suppose in your Genetics course you are asked to write a report about the "Hardy-Weinberg Equilibrium". Instead of you just copy-and-pasting sections of "CheatGPT" into a MS Word document, you could just tell your `Utimate Lab Partner` the report topic and its requirements, let her do all the rest.

Run the code in the next cell to see how well your `Ultimate Lab Partner` does...

In [ ]:
# Example 9A: The Ultimate Lab Partner

import google.generativeai as genai
from google.colab import userdata
from IPython.display import display, Markdown

# 1. SETUP
try:
    GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
except Exception as e:
    print("PLEASE CLICK 'GRANT ACCESS' IN THE POPUP!")
    print(f"Error: {e}")

# Define the model
model = genai.GenerativeModel('gemini-2.5-flash')

# 2. USER INPUT
topic = "Hardy-Weinberg Equilibrium"

# 3. STEP 1: THE EXPLANATION
print(f"1. Lab Partner is studying {topic}...")
explain_prompt = (
    f"Explain {topic} clearly for a biology undergraduate. "
    "Include the main formula."
)
explanation = model.generate_content(explain_prompt).text

# 4. STEP 2: THE VISUALIZATION (Code Gen)
print("2. Lab Partner is writing a simulation script...")
code_prompt = (
    f"Based on this explanation: '{explanation}', "
    "write a Python script using matplotlib to plot the allele frequencies (p and q) "
    "over time or in relation to genotype frequencies (p^2, 2pq, q^2). "
    "Provide ONLY the raw code inside a Markdown block."
)
code_response = model.generate_content(code_prompt).text

# 5. STEP 3: THE QUIZ
print("3. Lab Partner is writing a quiz question...")
quiz_prompt = (
    f"Based on {topic}, generate one difficult multiple-choice question "
    "to test the student's understanding. Do not give the answer yet."
)
quiz_question = model.generate_content(quiz_prompt).text

# 6. GENERATE FINAL REPORT
display(Markdown(f"# 🧬 Lab Partner Report: {topic}"))
display(Markdown("### Concept Summary"))
display(Markdown(explanation))
display(Markdown("---"))
display(Markdown("### Visualization Code"))
display(Markdown("Run this code in a new cell to see the graph!"))
display(Markdown(code_response))
display(Markdown("---"))
display(Markdown("### ❓ Pop Quiz"))
display(Markdown(quiz_question))

If the code is correct you should see something _similar_ to the following:

Here is the first part of the output, the lab report:

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image22B.png)


### Example 9B: Run the Code

The cell below contains the Python generated above by your `Ultimate Lab Partner`. Run the code to see the output.

In [ ]:
# Example 9B: Run the Code

import matplotlib.pyplot as plt
import numpy as np

# Create a range of allele frequencies for 'p'
# q will automatically be 1 - p
p_values = np.linspace(0, 1, 100)
q_values = 1 - p_values

# Calculate genotype frequencies based on p and q
p_squared = p_values**2
two_pq = 2 * p_values * q_values
q_squared = q_values**2

# Plotting
plt.figure(figsize=(10, 6))

# Plot allele frequencies
plt.plot(p_values, p_values, label='Frequency of Allele A (p)', linestyle='--')
plt.plot(p_values, q_values, label='Frequency of Allele a (q)', linestyle=':')

# Plot genotype frequencies
plt.plot(p_values, p_squared, label='Frequency of Genotype AA (p^2)', color='red')
plt.plot(p_values, two_pq, label='Frequency of Genotype Aa (2pq)', color='green')
plt.plot(p_values, q_squared, label='Frequency of Genotype aa (q^2)', color='blue')

# Add labels and title
plt.xlabel("Frequency of Allele A (p)")
plt.ylabel("Frequency")
plt.title("Hardy-Weinberg Equilibrium: Allele and Genotype Frequencies")
plt.legend()
plt.grid(True)
plt.axhline(1, color='gray', linestyle='--', linewidth=0.7) # Line at 1 for total frequency
plt.axhline(0, color='gray', linestyle='--', linewidth=0.7) # Line at 0 for total frequency
plt.ylim(0, 1.05) # Ensure y-axis covers all frequencies
plt.xlim(0, 1) # Ensure x-axis covers all p values

# Annotate the sum of genotype frequencies always equals 1
plt.text(0.5, 0.95, r'$p^2 + 2pq + q^2 = 1$', horizontalalignment='center', fontsize=12, bbox=dict(facecolor='white', alpha=0.7))
plt.text(0.5, 0.05, r'$p + q = 1$', horizontalalignment='center', fontsize=12, bbox=dict(facecolor='white', alpha=0.7))


plt.show()

If the code is correct you should see the followingoutput:

This is the second part of the output from your `Ultimate Lab Partner`.

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image21B.png)

And finally, here is the last part of the output, a Quiz Question that your `Ultimate Lab Partner` has generated for you.

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_4_image23B.png)

## **A Last Word from your Professor**

Very few predicted how fast AI would develop when I first started teaching BIO1173 and how AI would dramatically alter our world.

At present, this change is still accelerating and there no way to know exactly how your life will be impacted.

My hope that this course has helped to prepare you for that future, whatever it is.

Good luck!

Dr. David Senseman

## **Lesson Turn-in**

When you have completed and run all of the code cells, use the **File --> Print.. --> Save to PDF** to generate a PDF of your Colab notebook if you are using a Mac. If you are using a Windows computer, then use the **File --> Print.. --> Microsoft Print to PDF** to generate a PDF of your Colab notebook. Save your PDF as `Class_06_4.lastname.pdf` where _lastname_ is your last name, and upload the file to Canvas.

# **Lizard Tail**

## **Google NotebookLM**

### **The Undergraduate’s Guide to Google NotebookLM: Your New AI Research Partner**

![__](https://biologicslab.co/BIO1173/images/class_06/NotebookLM.png)


### **Introduction: The "Grounding" Revolution**

College life is defined by information overload. Between 50-page academic journal articles, hour-long lecture recordings, sprawling slide decks, and dense textbook chapters, the modern undergraduate is constantly drowning in data. The challenge isn't just *reading* it all; it's synthesizing it, understanding it, and turning it into arguments for papers or answers for exams.

### Enter **Google NotebookLM**.

Released broadly in late 2023 and evolving rapidly throughout 2024 and 2025, NotebookLM (Notebook Language Model) is distinct from the AI chatbots you likely use, such as ChatGPT or standard Gemini. While a standard chatbot is a "know-it-all" trained on the entire internet, NotebookLM is a "know-it-specifics." It is an **AI-powered research assistant grounded in your own documents.**

The core concept is **Source Grounding**. When you ask ChatGPT a question, it predicts the next word based on everything it has ever read on the web, which leads to "hallucinations" (confident lies). When you ask NotebookLM a question, it answers *only* using the documents you have uploaded. If the answer isn't in your Geology 101 slides, it will tell you it doesn't know.

For a college student, this is the difference between an AI that *guesses* and an AI that *cites*.

This guide will cover what NotebookLM is, how to access it, and four specific "playbooks" for using it to crush your semester.


### **Part 1: The Basics**

### What is it?
NotebookLM is a web-based interface that combines a note-taking app with a powerful Large Language Model (Gemini 1.5 Pro). You create "Notebooks"—think of them as separate folders for each of your classes or research projects. Inside each notebook, you upload sources. The AI then "reads" and "memorizes" every word in those sources, becoming an instant expert on that specific material.

### Key Features at a Glance
* **Multimodal Sources:** It reads almost anything. You can upload Google Docs, PDFs, Text files, and Google Slides. You can also paste text from your clipboard or copy links to websites. Uniquely, you can paste a **YouTube URL**, and it will read the transcript of the video.
* **Inline Citations:** Every time the AI makes a claim, it provides a gray citation number (e.g., `[1]`). Clicking this number jumps you immediately to the exact paragraph in your uploaded PDF or slide where the information came from.
* **Audio Overviews (The "Deep Dive"):** This is the viral feature that put NotebookLM on the map. With one click, two AI hosts (a "male" and "female" voice) generate a banter-filled, podcast-style conversation discussing your material. They use analogies, banter, and simplified language to explain complex topics.
* **Study Aids:** It can automatically generate **Flashcards**, **Quizzes**, **Timelines**, and **Briefing Docs** based on your sources.


### **Part 2: How to Access NotebookLM**

Accessing the tool is simple, but understanding the tier differences is helpful.

### The Free Version (Personal)
For 99% of undergraduates, the free version is all you need. It is incredibly generous compared to other AI tools.
* **How to access:** Go to [notebooklm.google.com](https://notebooklm.google.com) and sign in with your personal Google (Gmail) account.
* **Limits:** You can create up to 100 notebooks. Each notebook can hold 50 sources. Each source can be up to 500,000 words. (For context, *Harry Potter and the Sorcerer's Stone* is about 77,000 words. You can upload almost seven of them as a *single* source).
* **Cost:** $0.

### The "Business" Version (Workspace/Education)
If your university uses Google Workspace for Education, you may have access to the Enterprise/Education version.
* **Access:** Sign in with your `.edu` email address. Note that your university IT admin must enable "Early Access Apps" or "NotebookLM" specifically for students, so this may not work for everyone.
* **Benefits:** Enhanced data privacy (guarantees your data is not used to train Google's models—though Google states the personal version also doesn't train on your data unless you give feedback), and higher usage limits (up to 5x the sources).
* **NotebookLM Business:** There is also a paid "NotebookLM Business" tier for companies, offering team-shared notebooks and admin controls, but undergraduates rarely need to purchase this individually.

### **Part 3: The Engine Room – How It Works**

Once you log in, the interface is clean and minimal. Here is the workflow:

1.  **Create a Notebook:** Click the big "New Notebook" box. Title it "BIO 101" or "Thesis Research."
2.  **Add Sources:** On the left sidebar, click the `+` icon.
    * *Pro Tip:* You can upload your entire syllabus, all 12 weeks of lecture slides (export them to PDF first if they are in PowerPoint), and the course textbook chapters.
3.  **The "Guide" Window:** As soon as sources upload, NotebookLM pops up a "Notebook Guide." This offers auto-generated summaries.
4.  **The Chat Bar:** At the bottom is a chat box. This is where you interrogate your data.
    * *Example:* "Based on these 12 slide decks, what are the three most common causes of cell apoptosis mentioned by the professor?"
5.  **The Dashboard:** Above the chat, you will see "suggested actions" like *Help me study*, *Critique*, or *Suggest related ideas*.


### **Part 4: The Undergraduate Playbook (Use Cases)**

Here are four specific ways to use NotebookLM to survive college, ranked by common student needs.

### Use Case 1: The "Impossible" Reading List (Audio Overviews)
**The Problem:** You have 150 pages of Sociology readings assigned for tomorrow. You are exhausted, your eyes hurt, and you have a 30-minute commute to campus.
**The Solution:** The "Deep Dive" Audio Overview.

**Instructions:**
1.  Download your assigned readings (PDFs) and upload them to a new Notebook.
2.  Click the **"Notebook Guide"** button (if not already open).
3.  Click the **"Audio Overview"** button (sometimes labeled "Generate" under Audio).
4.  **Wait:** It takes about 3-5 minutes to process.
5.  **Listen:** The AI hosts will start a 10-15 minute conversation.
    * *Why this works:* They don't just read the text; they synthesize it. One host might ask, "So, Durkheim is basically saying society is like a body?" and the other will reply, "Exactly! That's functionalism. If one organ fails..."
    * *Pro Tip:* Use the **"Customize"** button before generating audio to guide the hosts. You can type: "Focus specifically on the contrast between Marx and Weber, and use analogies related to modern tech companies." The hosts will then tailor their banter to that instruction.

### Use Case 2: The "Needle in a Haystack" (Research Papers)
**The Problem:** You are writing a paper on "The impact of microplastics on coral reefs." You have downloaded 20 PDFs from JSTOR. You need to find every quote that mentions "bleaching," but you don't remember which paper said what.
**The Solution:** Cross-Source Querying with Citations.

**Instructions:**
1.  Upload all 20 PDFs to one Notebook.
2.  In the chat bar, type: *"List every instance where coral bleaching is linked to microplastic ingestion. Include direct quotes and citations."*
3.  **The Magic:** NotebookLM will output a bulleted list. Next to each point, you will see a small citation number like `[3]`.
4.  Click the `[3]`. The left panel will instantly scroll to the exact page in the exact PDF where that sentence exists.
    * *Why this works:* This prevents "citation anxiety." You know exactly where the info came from, so you can build your bibliography correctly. You are summarizing 20 papers at once without opening 20 tabs.

### Use Case 3: The "Final Exam" Cram Session
**The Problem:** The final is cumulative. You have a semester's worth of notes and slides. You need to test yourself, but you don't have time to make flashcards.
**The Solution:** Automated Study Guides.

**Instructions:**
1.  Create a "Final Exam" Notebook.
2.  Upload your class notes (Google Docs) and lecture slides.
3.  Look at the "Suggested Actions" chips above the chat bar.
4.  Click **"Study Guide"**.
    * It will generate a structured guide with key terms, essay questions, and summaries.
5.  Type in the chat: *"Create a 20-question multiple choice quiz based on the 'Midterm Review' source, but make it difficult."*
6.  Type: *"Create a glossary of all bolded terms in the slide decks."*

### Use Case 4: The Lecture "Decoder" (YouTube Integration)
**The Problem:** You missed a lecture, or you were in class but zoned out. The professor posted the lecture recording on YouTube, but it's 90 minutes long.
**The Solution:** The YouTube Transcript Processor.

**Instructions:**
1.  Copy the URL of the lecture video.
2.  In NotebookLM, click "Add Source" -> "Website" -> "YouTube".
3.  Paste the link.
4.  Ask: *"Summarize the professor's main argument in the last 20 minutes of class."* or *"Did the professor mention what will be on the test?"*
    * *Why this works:* It reads the transcript faster than you can watch at 2x speed. You can also ask it to "Compare what the professor said in this video to the textbook chapter I uploaded," finding discrepancies between lecture and text.


### **Part 5: Advanced Features & Tips**

### The "Learning Guide" Mode
Google recently added a "Learning Guide" feature. Instead of just answering your question, you can instruct NotebookLM to act as a **Tutor**.
* **Prompt:** *"Don't just give me the answer. Quiz me on the concepts in these notes. Ask me one question at a time and wait for my response. If I'm wrong, give me a subtle hint."*
* Because the AI has a large "context window" (memory), it can hold a sustained conversation, effectively roleplaying as a TA (Teaching Assistant).

### Shared Notebooks
If you are working on a Group Project, you can share a Notebook (similar to sharing a Google Doc).
* Click the "Share" icon in the top right.
* Add your group members' email addresses.
* **Note:** In the free version, check permissions carefully. Usually, all editors can add sources. This creates a "Group Brain" where if your partner finds a great article, they upload it, and *you* can immediately ask the AI questions about it.

### Privacy and Data Security
A major concern for students is privacy.
* **Personal Data:** Google explicitly states that for NotebookLM, **your data is NOT used to train the model** unless you explicitly provide feedback (thumbs up/down and submit logs). This is a higher standard of privacy than the free version of ChatGPT.
* **Academic Integrity:** Do not use NotebookLM to *write* your paper. Use it to *research* your paper. If you copy-paste the output, it may be detected by AI checkers, but more importantly, it will lack your personal voice. The "Citation" feature is your safety net—always click the citation and read the original source text before using it in your work to ensure the AI didn't misinterpret the context.

### **Part 6: Limitations (The "Fine Print")**

While impressive, NotebookLM is not perfect. Undergrads should be aware of these traps:
1.  **The "Silo" Effect:** NotebookLM *only* knows what you upload. If you upload an outdated article, it will give you outdated answers. It does not "fact check" your sources against the live internet (unless you enable specific web features, but generally it is grounded in your uploads). Garbage in, garbage out.
2.  **Audio "Hallucinations":** The Audio Overview hosts are incredibly realistic, but they can sometimes invent "banter" that implies facts not in the text. For example, they might joke, "Oh, like that time in 1999..." when the text never mentioned 1999. Use the audio for concepts, not for precise dates or quotes.
3.  **Image Blindness:** While it can read text *in* slides, it sometimes struggles with complex charts or unlabelled graphs inside PDFs. Always verify data from charts manually.

### **Conclusion**

Google NotebookLM represents a shift from "Search" to "Synthesis." In the past, the hard part of college was finding information. Today, the hard part is filtering it. By creating a private AI that knows your syllabus, your readings, and your notes, you effectively build a "Second Brain" that frees you up to do the actual thinking.

Start with one class. Upload your syllabus and the first week's readings. Generate an Audio Overview for your walk to class. You may find that for the first time, you aren't just surviving the reading list—you're mastering it.